In [62]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

# 一、读取日度行情表

表内字段就是 Backtrader 默认情况下要求输入的 7 个字段： 'datetime' 、'open'、'high'、'low'、'close'、'volume'、'openinterest'，外加一个 'sec_code' 股票代码字段。

In [2]:
daily_price = pd.read_csv("./data/daily_price.csv", parse_dates=['datetime'])
daily_price

,datetime,sec_code,open,high,low,close,volume,openinterest
0,2019-01-02,600466.SH,33.064891,33.496709,31.954503,32.386321,10629352,0
1,2019-01-02,603228.SH,50.660230,51.458513,50.394136,51.120778,426147,0
2,2019-01-02,600315.SH,148.258423,150.480132,148.258423,149.558935,2138556,0
3,2019-01-02,000750.SZ,49.512579,53.154883,48.715825,51.561375,227557612,0
4,2019-01-02,002588.SZ,36.608672,36.608672,35.669988,35.763857,2841517,0
...,...,...,...,...,...,...,...,...
255967,2021-01-28,600717.SH,121.489201,122.011736,120.705400,120.966667,6022213,0
255968,2021-01-28,300558.SZ,134.155888,137.600704,130.700970,131.569750,5330301,0
255969,2021-01-28,600171.SH,39.774873,39.830040,38.864630,38.947380,12354183,0
255970,2021-01-28,600597.SH,47.190201,49.243025,46.250355,46.423484,32409940,0


In [3]:
# 筛选 600466.SH 和 603228.SH 2只股票的数据集
data1 = daily_price.query(f"sec_code=='600466.SH'").set_index('datetime').drop(columns=['sec_code'])
data2 = daily_price.query(f"sec_code=='603228.SH'").set_index('datetime').drop(columns=['sec_code'])
data2

,open,high,low,close,volume,openinterest
datetime,,,,,,
2019-01-02,50.660230,51.458513,50.394136,51.120778,426147,0
2019-01-03,50.609059,51.049137,50.107573,50.639762,492071,0
2019-01-04,50.199683,51.171950,49.278588,50.455543,665486,0
2019-01-07,50.864918,51.417575,50.353199,50.967262,689444,0
2019-01-08,51.110544,52.174920,50.250855,50.527183,931211,0
...,...,...,...,...,...,...
2021-01-22,59.955312,59.955312,57.478496,58.174461,6410959,0
2021-01-25,58.113052,58.133522,57.110045,57.498966,3445027,0
2021-01-26,57.498966,57.498966,54.305716,55.124498,7340180,0


# 二、self.datas 测试

In [64]:
class TestStrategy(bt.Strategy):
    
    def __init__(self):
        # 打印数据集和数据集对应的名称
        print("-------------self.datas-------------")
        print(self.datas)
        print("-------------self.data-------------")
        print(self.data._name, self.data)
        print("-------------self.data0-------------")
        print(self.data0._name, self.data0)
        print("-------------self.datas[0]-------------")
        print(self.datas[0]._name, self.datas[0])
        print("-------------self.datas[1]-------------")
        print(self.datas[1]._name, self.datas[1])
        print("-------------self.datas[-1]-------------")
        print(self.datas[-1]._name, self.datas[-1])
        print("-------------self.datas[-2]-------------")
        print(self.datas[-2]._name, self.datas[-2])
        
cerebro1 = bt.Cerebro()

datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed1, name='600466.SH')
datafeed2 = bt.feeds.PandasData(dataname=data2, fromdate=datetime.datetime(2019,1,2), todate=datetime.datetime(2021,1,28))
cerebro1.adddata(datafeed2, name='603228.SH')

cerebro1.addstrategy(TestStrategy)
rasult = cerebro1.run()

-------------self.datas-------------
[<backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a220>, <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a820>]
-------------self.data-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a220>
-------------self.data0-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a220>
-------------self.datas[0]-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a220>
-------------self.datas[1]-------------
603228.SH <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a820>
-------------self.datas[-1]-------------
603228.SH <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a820>
-------------self.datas[-2]-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x7f6a3f14a220>


# 三、lines 测试

## 3.1 如何调用某一条 line ?

In [65]:
class TestStrategy(bt.Strategy):
 
    def __init__(self):
        print("--------- 打印 self 策略本身的 lines ----------")
        print(self.lines.getlinealiases())
        print("--------- 打印 self.datas 第一个数据表格的 lines ----------")
        print(self.datas[0].lines.getlinealiases())
        # 计算第一个数据集的s收盘价的20日均线，返回一个 Data feed
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0].close, period=20)
        print("--------- 打印 indicators 对象的 lines ----------")
        print(self.sma.lines.getlinealiases())
        print("---------- 直接打印 indicators 对象的所有 lines -------------")
        print(self.sma.lines) 
        print("---------- 直接打印 indicators 对象的第一条 lines -------------")
        print(self.sma.lines[0])
        
    def next(self):
        print('验证索引位置为 6 的线是不是 datetime')
        print(bt.num2date(self.datas[0].lines[6][0]))
        # num2date() 作用是将数字形式的时间转为 date 形式
        
cerebro2 = bt.Cerebro()
st_date = datetime.datetime(2019,1,2)
ed_date = datetime.datetime(2021,1,28)
datafeed1 = bt.feeds.PandasData(dataname=data1, 
                                fromdate=st_date, 
                                todate=ed_date)
cerebro2.adddata(datafeed1, name='600466.SH')
datafeed2 = bt.feeds.PandasData(dataname=data2, 
                                fromdate=st_date,         
                                todate=ed_date)
cerebro2.adddata(datafeed2, name='603228.SH')

cerebro2.addstrategy(TestStrategy)
rasult = cerebro2.run()

--------- 打印 self 策略本身的 lines ----------
('datetime',)
--------- 打印 self.datas 第一个数据表格的 lines ----------
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime')
--------- 打印 indicators 对象的 lines ----------
('sma',)
---------- 直接打印 indicators 对象的所有 lines -------------
---------- 直接打印 indicators 对象的第一条 lines -------------
验证索引位置为 6 的线是不是 datetime
2019-01-29 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-01-30 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-01-31 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-01 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-11 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-12 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-13 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-14 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-15 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-18 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-19 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-20 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-21 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-22 00:00:00
验证索引位置为 6 的线是不是 datetime
2019-02-

## 3.2 如何提取 line 上的数据点？

In [66]:
cerebro3 = deepcopy(cerebro2)

class TestStrategy(bt.Strategy):
    def __init__(self):
        self.count = 0 # 用于计算 next 的循环次数
        # 打印数据集和数据集对应的名称
        print("------------- init 中的索引位置-------------")
        print("0 索引：",'datetime',self.data1.lines.datetime.date(0), 'close',self.data1.lines.close[0])
        print("-1 索引：",'datetime',self.data1.lines.datetime.date(-1),'close', self.data1.lines.close[-1])
        print("-2 索引",'datetime', self.data1.lines.datetime.date(-2),'close', self.data1.lines.close[-2])
        print("1 索引：",'datetime',self.data1.lines.datetime.date(1),'close', self.data1.lines.close[1])
        print("2 索引",'datetime', self.data1.lines.datetime.date(2),'close', self.data1.lines.close[2])
        print("从 0 开始往前取3天的收盘价：", self.data1.lines.close.get(ago=0, size=3))
        print("从-1开始往前取3天的收盘价：", self.data1.lines.close.get(ago=-1, size=3))
        print("从-2开始往前取3天的收盘价：", self.data1.lines.close.get(ago=-2, size=3))
        print("line的总长度：", self.data1.buflen())
        
    def next(self):
        print(f"------------- next 的第{self.count+1}次循环 --------------")
        print("当前时点（今日）：",'datetime',self.data1.lines.datetime.date(0),'close', self.data1.lines.close[0])
        print("往前推1天（昨日）：",'datetime',self.data1.lines.datetime.date(-1),'close', self.data1.lines.close[-1])
        print("往前推2天（前日）", 'datetime',self.data1.lines.datetime.date(-2),'close', self.data1.lines.close[-2])
        print("前日、昨日、今日的收盘价：", self.data1.lines.close.get(ago=0, size=3))
        print("往后推1天（明日）：",'datetime',self.data1.lines.datetime.date(1),'close', self.data1.lines.close[1])
        print("往后推2天（明后日）", 'datetime',self.data1.lines.datetime.date(2),'close', self.data1.lines.close[2])
        print("已处理的数据点：", len(self.data1))
        print("line的总长度：", self.data1.buflen())
        self.count += 1
        
cerebro3.addstrategy(TestStrategy)
rasult = cerebro3.run()

--------- 打印 self 策略本身的 lines ----------
('datetime',)
--------- 打印 self.datas 第一个数据表格的 lines ----------
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime')
--------- 打印 indicators 对象的 lines ----------
('sma',)
---------- 直接打印 indicators 对象的所有 lines -------------
---------- 直接打印 indicators 对象的第一条 lines -------------
------------- init 中的索引位置-------------
0 索引： datetime 2021-01-28 close 54.91980265
-1 索引： datetime 2021-01-27 close 55.5952978
-2 索引 datetime 2021-01-26 close 55.12449815
1 索引： datetime 2019-01-02 close 51.12077805
2 索引 datetime 2019-01-03 close 50.63976172
从 0 开始往前取3天的收盘价： array('d')
从-1开始往前取3天的收盘价： array('d', [57.49896595, 55.12449815, 55.5952978])
从-2开始往前取3天的收盘价： array('d', [58.1744611, 57.49896595, 55.12449815])
line的总长度： 506
------------- next 的第1次循环 --------------
当前时点（今日）： datetime 2019-01-02 close 51.12077805
往前推1天（昨日）： datetime 2021-01-28 close 54.91980265
往前推2天（前日） datetime 2021-01-27 close 55.5952978
前日、昨日、今日的收盘价： array('d')
往后推1天（明日）： datetime

IndexError: array index out of range

# 四、自定义读入函数

In [67]:
class My_CSVData(bt.feeds.GenericCSVData):
    params = (
    ('fromdate', datetime.datetime(2019,1,2)),
    ('todate', datetime.datetime(2021,1,28)),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d')),
    ('datetime', 0),
    ('time', -1),
    ('high', 3),
    ('low', 4),
    ('open', 2),
    ('close', 5),
    ('volume', 6),
    ('openinterest', -1)
)

cerebro4 = bt.Cerebro()
data = My_CSVData(dataname='daily_price.csv')
cerebro4.adddata(data, name='600466.SH')
rasult = cerebro4.run()

# 五、新增指标

In [69]:
class PandasData_more(bt.feeds.PandasData):
    lines = ('pe', 'pb', )  # 要添加的线
    # 设置 line 在数据源上的列位置
    params=(
        ('pe', -1),
        ('pb', -1),
           ) 
    # -1表示自动按列明匹配数据，也可以设置为线在数据源中列的位置索引 (('pe',6),('pb',7),) 
class TestStrategy(bt.Strategy):
    def __init__(self):
        print("--------- 打印 self.datas 第一个数据表格的 lines ----------")
        print(self.data0.lines.getlinealiases())
        print("pe line:", self.data0.lines.pe)
        print("pb line:", self.data0.lines.pb)

data1['pe'] = 2 # 给原先的data1新增pe指标（简单的取值为2）
data1['pb'] = 3 # 给原先的data1新增pb指标（简单的取值为3）
# 导入的数据 data1 中
cerebro5 = bt.Cerebro()
st_date = datetime.datetime(2019,1,2)
ed_date = datetime.datetime(2021,1,28)
datafeed1 = PandasData_more(dataname=data1, 
                            fromdate=st_date, 
                            todate=ed_date)
cerebro5.adddata(datafeed1, name='600466.SH')
cerebro5.addstrategy(TestStrategy)
rasult = cerebro5.run()

--------- 打印 self.datas 第一个数据表格的 lines ----------
('close', 'low', 'high', 'open', 'volume', 'openinterest', 'datetime', 'pe', 'pb')
pe line: <backtrader.linebuffer.LineBuffer object at 0x7f6a3f1234c0>
pb line: <backtrader.linebuffer.LineBuffer object at 0x7f6a3f123520>
